In [154]:
import pandas as pd
import numpy as np

#### Пункт 1. Считываем файлы

In [203]:
first_file = pd.read_csv("2.tmp",encoding='utf-8', sep='\n', header=None)
second_file = pd.read_csv("3.tmp", encoding='utf-8', sep='\n', header=None)
third_file = pd.read_csv("4.tmp", encoding='utf-8', sep='\n', header=None)

Посмотрим, что у нас получилось

In [204]:
first_file.head()

,0
0,Bag End先生的Bilbo Baggins先生宣布，他不久將以特別的輝煌的聚會慶祝他的第...
1,NaN
2,NaN
3,什麼，人們普遍相信，Bag End山充滿了寶藏的隧道。如果這還不夠名聲，那麼他也有長期的活力...
4,NaN


Делаем препроцессинг для файлов

In [205]:
def preprocess_data(file, file_num):
    # Переименовываем колонки
    file.columns = ['line']
    # Добавляем колонку target - номер файла
    file['target'] = [file_num]*len(file)
    return file

In [206]:
first_file = preprocess_data(first_file, 1)
second_file = preprocess_data(second_file, 2)
third_file = preprocess_data(third_file, 3)

#### Пункт 2. Соединяем файлы и отчищаем от значений NaN

In [207]:
data = pd.concat([first_file, second_file, third_file], ignore_index=True)
data.dropna(inplace=True)
data.head()

,line,target
0,Bag End先生的Bilbo Baggins先生宣布，他不久將以特別的輝煌的聚會慶祝他的第...,1
3,什麼，人們普遍相信，Bag End山充滿了寶藏的隧道。如果這還不夠名聲，那麼他也有長期的活力...,1
6,青年以及（著名地）無窮無盡的財富似乎是不公平的。他們說：“這將需要支付。” “這不是自然的，...,1
8,仍然與親屬（除了當然是薩克維爾 - 巴金斯）的訪問條件，他在貧窮和不重要的家庭的愛好中有許多...,1
30,＃＃”你說得對，爸爸！“Gaffer說。 “不是巴剋土地的白蘭地酒在老森林裡生活，但他們似乎...,1


In [208]:
data.index = range(len(data))
data.head()

,line,target
0,Bag End先生的Bilbo Baggins先生宣布，他不久將以特別的輝煌的聚會慶祝他的第...,1
1,什麼，人們普遍相信，Bag End山充滿了寶藏的隧道。如果這還不夠名聲，那麼他也有長期的活力...,1
2,青年以及（著名地）無窮無盡的財富似乎是不公平的。他們說：“這將需要支付。” “這不是自然的，...,1
3,仍然與親屬（除了當然是薩克維爾 - 巴金斯）的訪問條件，他在貧窮和不重要的家庭的愛好中有許多...,1
4,＃＃”你說得對，爸爸！“Gaffer說。 “不是巴剋土地的白蘭地酒在老森林裡生活，但他們似乎...,1


#### Пункт 3. Поробуем предсказывать метки классов для каждой записи, используя логистическую регрессию, в качестве признаков будем использовать n-граммы

In [210]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

В качестве признаков будем брать символьные n-граммы от 3 до 5 символов

In [211]:
vectorizer = CountVectorizer(ngram_range=(3,5), analyzer='char_wb', lowercase=False, max_df=0.84, binary=True)

Получаем разреженную матрицу, где в качестве признаков будет наличие или отсутствие определённой n-граммы

In [212]:
%%time
data_matrix = vectorizer.fit_transform(data['line'])

Wall time: 658 ms


In [213]:
data_matrix

<1943x133091 sparse matrix of type '<class 'numpy.int64'>'
	with 205262 stored elements in Compressed Sparse Row format>

#### Пункт 4. Оцениваем результат, используя KFold и accuracy_score

In [214]:
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score

In [215]:
%%time
score = []
predictions = []
# Используйте процедуру KFold для проверки качества классификаторов, не забываем перемешивать данные
kf = KFold(len(data), n_folds=10, shuffle=True)
for train_index, test_index in kf:
    X_train, X_test = data_matrix[train_index], data_matrix[test_index]
    y_train, y_test = data['target'][train_index], data['target'][test_index]
    
    model = LogisticRegression(penalty='l1', n_jobs=-1, random_state=45)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    score.append(accuracy_score(y_test, predictions))

Wall time: 4.84 s


Посмотрим на результат классификации

In [216]:
print("Accuracy scores: {}\n\n".format(score))
print("Mean accuracy score: {}".format(np.array(score).mean()))

Accurasy scores: [0.94358974358974357, 0.96923076923076923, 0.95897435897435901, 0.95360824742268047, 0.96391752577319589, 0.98453608247422686, 0.95360824742268047, 0.96391752577319589, 0.97422680412371132, 0.95360824742268047]


Mean accurasy score: 0.9619217552207242


Точность достаточно высокая, выше 0.85, чего мы и добивались

#### Пункт 5. Поробуем определить языки с помощью модуля unicodedata

In [220]:
import unicodedata
from string import punctuation, whitespace, digits

In [221]:
def get_language_code(string):
    lang = []
    for ch in string:
        if (ch not in punctuation and ch not in whitespace and ch not in digits):
            lang.append(unicodedata.name(ch).split(" ")[0])
    lang = np.unique(lang)
    return lang

In [222]:
first_file_lang = get_language_code(first_file['line'][0])
second_file_lang = get_language_code(second_file['line'][0])
third_file_lang = get_language_code(third_file['line'][0])

In [223]:
print("Languages in first file: {}".format(first_file_lang))
print("Languages in second file: {}".format(second_file_lang))
print("Languages in third file: {}".format(third_file_lang))

Languages in first file: ['CJK' 'FULLWIDTH' 'LATIN']
Languages in second file: ['LATIN']
Languages in third file: ['CJK' 'HIRAGANA' 'IDEOGRAPHIC' 'KATAKANA' 'KATAKANA-HIRAGANA' 'LATIN']


Осталось только посмотреть, что означают данные кодировки
* CJK - Китайский или корейский или японский
* LATIN - понятно)
* HIRAGANA - видимо какая-то разновидность японского
* KATAKANA - какая-то разновидность японского
* KATAKANA-HIRAGANA - микс из японских письменностей)